<a href="https://colab.research.google.com/github/Andrezaga/Agregador-de-Dados-Econ-micos/blob/main/DadosAutom%C3%A1ticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Extração de dados dos indicadores e inserção na planilha de análise

# Bibliotecas


In [ ]:
import pandas as pd
import numpy as np
from pandas_datareader import data as web
from time import sleep
from datetime import datetime
from openpyxl import Workbook, load_workbook

import requests 
from bs4 import BeautifulSoup

!pip install selenium
!apt-get update
!apt install chromium-chromedriver

from selenium import webdriver

!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
yf.pdr_override()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:15 http://archive.ubuntu.com/ubun

# Funções



In [ ]:
#Faz a consulta dos dados do Bacen
def consulta_bc(codigo_bcb):
  url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo_bcb)
  df = pd.read_json(url)
  df.columns = ['Data','Valor']
  return df

In [ ]:
#Faz a consulta dos dados do Yahoo Finance
#def consulta_yf(codigo_yf):
#df = web.DataReader('^BVSP', data_source='yahoo')
#df
  #return df

#PROBLEMA

In [ ]:
#Faz a consulta dos dados do Yahoo Finance
def consulta_yf(codigo_yf):
  ativo = web.get_data_yahoo(codigo_yf)
  return ativo

In [ ]:
#Extrai dados de uma página do IpeaData
def consulta_riscobrasil(dias):
  ipea = requests.get('http://www.ipeadata.gov.br/ExibeSerie.aspx?serid=40940&module=M')
  ipeahtml = ipea.content
  siteipea = BeautifulSoup(ipeahtml,'html.parser')
  tabelaipea = siteipea.find('table', attrs={'id':'grd_DXMainTable'})
  data = []
  valor = []
  for i in np.arange(dias):
    text = 'grd_DXDataRow{}'.format(i)
    riskBR = tabelaipea.find('tr', attrs={'id': text})
    a = str(riskBR.text)
    data.append(a[:11].strip())
    valor.append(a[11:].strip())
  df = pd.DataFrame(list(zip(data,valor)))
  df.columns=['Data','Valor']
  df.Valor = pd.to_numeric(df.Valor)
  return df

In [ ]:
#Extrai dados de uma página do Yahii
def consultavar_iie2022(mes):
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  wd_chrome = webdriver.Chrome(chrome_options=chrome_options)
  wd_chrome.get('http://www.yahii.com.br/iieBR.html')
  tab = wd_chrome.find_element_by_xpath('/html/body/div/center/center/center[2]/center/center/center/font/font/div[3]/table')
  xpath = '/html/body/div/center/center/center[2]/center/center/center/font/font/div[3]/table/tbody/tr[{}]/td[8]'.format(mes)
  datatab1 = tab.find_element_by_xpath(xpath)
  datatab1.text
  if datatab1.text == ' ':
    mes -= 1
    xpath = '/html/body/div/center/center/center[2]/center/center/center/font/font/div[3]/table/tbody/tr[{}]/td[8]'.format(mes)
    datatab1 = tab.find_element_by_xpath(xpath)
  iien = float(datatab1.text.replace(',','.'))
  iien
  if mes == 2:
    xpath = '/html/body/div/center/center/center[2]/center/center/center/font/font/div[3]/table/tbody/tr[13]/td[7]'
    datatab2 = tab.find_element_by_xpath(xpath)
    iieb = float(datatab2.text.replace(',','.'))
  else:
    mes -= 1
    xpath = '/html/body/div/center/center/center[2]/center/center/center/font/font/div[3]/table/tbody/tr[{}]/td[8]'.format(mes)
    datatab2 = tab.find_element_by_xpath(xpath)
    iieb = float(datatab2.text.replace(',','.'))
  return iien, iieb                


# INPUTS

In [ ]:
lista_var = []

In [ ]:
#Pib mensal a valores correntes / valores e variação
PIB = consulta_bc(4380)
PIB = PIB.iloc[-2:,:]
PIBvar = (PIB.iloc[-1,1] - PIB.iloc[-2,1])/PIB.iloc[-2,1]
print(PIB)
lista_var.append(PIBvar)
PIBvar

           Data     Valor
382  01/11/2021  749729.4
383  01/12/2021  758918.9


0.012257089024386665

In [ ]:
#IBC-Br mensal / valores e variação
IBC = consulta_bc(24363)
IBC = IBC.iloc[-2:,:]
IBCvar = (IBC.iloc[-1,1] - IBC.iloc[-2,1])/IBC.iloc[-2,1]
print(IBC)
lista_var.append(IBCvar)
IBCvar

           Data   Valor
226  01/11/2021  139.76
227  01/12/2021  141.46


0.012163709215798634

In [ ]:
#ICB-Br mensal / valores e variação
ICB = consulta_bc(27574)
ICB = ICB.iloc[-2:,:]
ICBvar = (ICB.iloc[-1,1] - ICB.iloc[-2,1])/ICB.iloc[-2,1]
print(ICB)
lista_var.append(ICBvar)
ICBvar

           Data   Valor
287  01/12/2021  396.96
288  01/01/2022  408.84


0.02992744860943167

In [ ]:
#IBOV-Br mensal / valores e variação
IBOV = consulta_yf('^BVSP')
IBOV['Data'] = IBOV.index
str(IBOV.iloc[0,6])
IBOV
IBOVlst = int(str(IBOV.Data.iloc[-1])[5:7]) 
for i in range(-1,-32,-1):
  if IBOVlst - int(str(IBOV.Data.iloc[i])[5:7]) == 1:
    IBOVpass = IBOV.iloc[i,:]
    break
  else:
    continue
IBOVlst = IBOV.iloc[-1]
print(IBOVlst)
print(IBOVpass)
IBOVvar = (IBOVlst['Adj Close'] - IBOVpass['Adj Close'])/IBOVpass['Adj Close']
lista_var.append(IBOVvar)
IBOVvar


[*********************100%***********************]  1 of 1 completed
Open               112891.796875
High               113721.382812
Low                111888.210938
Close                112097.3125
Adj Close            112097.3125
Volume                         0
Data         2022-02-23 00:00:00
Name: 2022-02-23 00:00:00, dtype: object
Open                    111910.0
High                    112495.0
Low                     111195.0
Close                   112388.0
Adj Close               112388.0
Volume                  12190800
Data         2022-01-31 00:00:00
Name: 2022-01-31 00:00:00, dtype: object


-0.0025864638573513186

In [ ]:
#SELIC mensal / valores e variação
SELIC = consulta_bc(432)
SELIClst = int(SELIC.Data.iloc[-1][3:5]) 
for i in range(-1,-32,-1):
  if SELIClst - int(SELIC.Data.iloc[i][3:5]) == 1:
    SELICpass = SELIC.iloc[i,:]
    break
  else:
    continue
SELIClst = SELIC.iloc[-1]
print(SELIClst)
print(SELICpass)
SELICvar = (SELIClst.Valor - SELICpass.Valor)/SELICpass.Valor
lista_var.append(SELICvar)
SELICvar

Data     16/03/2022
Valor         10.75
Name: 8412, dtype: object
Data     28/02/2022
Valor         10.75
Name: 8396, dtype: object


0.0

In [ ]:
#IGP-M mensal / valores e variação
IGPM = consulta_bc(189)
IGPM = IGPM.iloc[-2:,:]
IGPMvar = (IGPM.iloc[-1,1] - IGPM.iloc[-2,1])/IGPM.iloc[-2,1]
print(IGPM)
lista_var.append(IGPMvar)
IGPMvar

           Data  Valor
390  01/12/2021   0.87
391  01/01/2022   1.82


1.0919540229885059

In [ ]:
#Dólar mensal / valores e variação
DOLAR = consulta_bc(3696)
DOLAR = DOLAR.iloc[-2:,:]
DOLARvar = (DOLAR.iloc[-1,1] - DOLAR.iloc[-2,1])/DOLAR.iloc[-2,1]
print(DOLAR)
lista_var.append(DOLARvar)
DOLARvar

           Data   Valor
827  01/12/2021  5.5805
828  01/01/2022  5.3574


-0.03997849655048824

In [ ]:
#Risco Brasil mensal / valores e variação
RISKB = consulta_riscobrasil(30)
RISKBlst = int(RISKB.Data.iloc[0][3:5]) 
for i in np.arange(32):
  if RISKBlst - int(RISKB.Data.iloc[i][3:5]) == 1:
    RISKBpass = RISKB.iloc[i,:]
    break
  else:
    continue
RISKBlst = RISKB.iloc[0]
print(RISKBlst)
print(RISKBpass)
RISKBvar = (RISKBlst.Valor - RISKBpass.Valor)/RISKBpass.Valor
lista_var.append(RISKBvar)
RISKBvar

Data     18/02/2022
Valor           340
Name: 0, dtype: object
Data     31/01/2022
Valor           334
Name: 14, dtype: object


0.017964071856287425

In [ ]:
#IIE-Br mensal / valores e variação
m = datetime.today()
m = int(str(m)[5:7])
IIEn,IIEb = consultavar_iie2022(m)
IIEvar = (IIEn - IIEb)/IIEb
print(IIEn)
print(IIEb)
lista_var.append(IIEvar)
IIEvar

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:393: UserWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  warnings.warn("find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead")


122.1
122.3


-0.00163532297628784

In [ ]:
#Produção Industrial mensal / valores e variação
PRODIND = consulta_bc(21859)
PRODIND = PRODIND.iloc[-2:,:]
PRODINDvar = (PRODIND.iloc[-1,1] - PRODIND.iloc[-2,1])/PRODIND.iloc[-2,1]
print(PRODIND)
lista_var.append(PRODINDvar)
PRODINDvar

           Data  Valor
238  01/11/2021   86.2
239  01/12/2021   79.2


-0.08120649651972157

In [ ]:
#Exportações mensal / valores e variação
EXPORT = consulta_bc(22711)
EXPORT = EXPORT.iloc[-2:,:]
EXPORTvar = (EXPORT.iloc[-1,1] - EXPORT.iloc[-2,1])/EXPORT.iloc[-2,1]
print(EXPORT)
lista_var.append(EXPORTvar)
EXPORTvar

           Data    Valor
323  01/12/2021  24147.1
324  01/01/2022  19425.0


-0.19555557396126239

In [ ]:
#Importações mensal / valores e variação
IMPORT = consulta_bc(22712)
IMPORT = IMPORT.iloc[-2:,:]
IMPORTvar = (IMPORT.iloc[-1,1] - IMPORT.iloc[-2,1])/IMPORT.iloc[-2,1]
print(IMPORT)
lista_var.append(IMPORTvar)
IMPORTvar

           Data    Valor
323  01/12/2021  21915.0
324  01/01/2022  21335.3


-0.02645220168834135

In [ ]:
#Desemprego mensal / valores e variação
DESMP = consulta_bc(24369)
DESMP = DESMP.iloc[-2:,:]
DESMPvar = (DESMP.iloc[-1,1] - DESMP.iloc[-2,1])/DESMP.iloc[-2,1]
print(DESMP)
lista_var.append(DESMPvar)
DESMPvar

           Data  Valor
115  01/10/2021   12.1
116  01/11/2021   11.6


-0.04132231404958678

In [ ]:
#Renda Média mensal / valores e variação
RENDM = consulta_bc(24381)
RENDM = RENDM.iloc[-2:,:]
RENDMvar = (RENDM.iloc[-1,1] - RENDM.iloc[-2,1])/RENDM.iloc[-2,1]
print(RENDM)
lista_var.append(RENDMvar)
RENDMvar

           Data  Valor
115  01/09/2021   2483
116  01/10/2021   2450


-0.013290374546919049

# Agregando as Variações



In [ ]:
# Variações dos dados mensais mais recentes dos indicadores
lista_var

[0.012257089024386665,
 0.012163709215798634,
 0.02992744860943167,
 -0.0025864638573513186,
 0.0,
 1.0919540229885059,
 -0.03997849655048824,
 0.017964071856287425,
 -0.00163532297628784,
 -0.08120649651972157,
 -0.19555557396126239,
 -0.02645220168834135,
 -0.04132231404958678,
 -0.013290374546919049]

# Jogando os dados na planilha de análise


In [ ]:
#Ajustando meses com as colunas do excel
mes_col = ['CTRL','D','E','F','G','H','I','J','K','L','M','N','O']
m = datetime.today()
m = int(str(m)[5:7])

In [ ]:
#Dados da variação dos indicadores na coluna do mês atual

col = mes_col[m]
col

wb = load_workbook('ModeloAnalise.xlsx')
ws = wb.active
ws = wb['Tabela de Tendências Nota']

cont = 0
contlist = 0
for i in ws[col]:
  cont += 1
  if cont > 4 and cont < 19:
    i.value = lista_var[contlist]
    contlist += 1
  elif cont == 18:
    break

ws = wb['Tendência para o mês']

ws['D19'] = m
ws['D20'] = col

cont=0
for i in ws['G']:
  cont 
wb.save(f'Tendência Setorial 2022.xlsx')